In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import xgboost as xgb
#https://habrahabr.ru/company/ods/blog/327242/

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
%matplotlib inline
import seaborn as sb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler


from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import __version__
from plotly import graph_objs as go

C:\Users\TEMP\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Формируем выборку временного ряда

In [2]:
df=pd.read_pickle('MG_Sales.pickle',compression='gzip')
celebrate=pd.read_pickle('celebrate.pickle')
#df['Дата']=np.array(df['Дата'], dtype='datetime64[M]')

## Формируем выборку

In [4]:
begin_period=dt.datetime(2015,1,1)
prediction_period=dt.datetime(2017,5,1)


#Пустой период в днях обучающей выборки
delta=prediction_period-begin_period
dummy_train = pd.DataFrame(index=np.array([begin_period + dt.timedelta(days=x) for x in range(0, delta.days)]).astype('datetime64[D]'))
dummy_train.index.name='ds'

#last_d=begin_period-dt.timedelta(days=30)
#(df['Магазин']=='Пермь (Колизей)  Ефименко О.Г.')&
#&(df['Дата']<prediction_period)prediction_period

#  and Магазин=='Варшавский' and ТоварЦеноваяГруппа=='18000-23000'  and Коллекция=='Аметисты с фианитами' and ЦветМеталла=='Красное золото'
#ТоварЦеноваяГруппа
#0-2100 
time_series=pd.DataFrame(data=df.query("Дата>=20150101 and ТоварЦеноваяГруппа=='2100-3200'").groupby('Дата')['Количество'].sum())
time_series.index.name='ds'
time_series.columns=['y']
time_series.y=time_series.y
#сливаем обучающую выборку и пустой период чтобы избежать пропусков дат, пропуски заполняем нулями
time_series=dummy_train.merge(time_series,left_index=True, right_index=True,how='outer').fillna(0)  

#time_series=time_series.loc[:prediction_period-dt.timedelta(seconds=1)]
y_test=time_series.loc[prediction_period:,'y']


date_list = np.array([prediction_period + dt.timedelta(days=x) for x in range(0, 30+31)]).astype('datetime64[D]')
time_series_forecast=pd.DataFrame(index=date_list)
time_series_forecast.index.name='ds'

## Фомируем характеристики модели

In [5]:
#Фиксируем аномально низкие и высокие продажи
ul=5#Персентиль высоких продаж 2
ll=5#7#Персентиль низких продаж 10
md=15#ширина медианы

ulim=np.percentile(time_series['y'], 100.-ul)
llim=np.percentile(time_series['y'], ll)
med=np.percentile(time_series['y'], [50-md,50+md])


time_series['Квантили']=0
time_series.loc[time_series.y<med[0],'Квантили']=-1
time_series.loc[time_series.y>med[1],'Квантили']=1
time_series.loc[time_series.y<llim,'Квантили']=-2
time_series.loc[time_series.y>ulim,'Квантили']=2


#вычисляем год назад
def yearsago(years, from_date):
    try:
        return from_date.replace(year=from_date.year - years)
    except ValueError:        
        return from_date.replace(month=2, day=28,
                                 year=from_date.year-years)

#временные характеристики
def setNewValues(time_series,celebrate):
    time_series['День недели'] = time_series.index.weekday
    time_series['Неделя'] = time_series.index.week
    time_series['Год'] = time_series.index.year
    time_series['День месяца'] = time_series.index.day
    time_series['День года'] = time_series.index.dayofyear
    time_series['Праздник']=0#Обычный день
    time_series.loc[time_series.index.isin(celebrate['Праздник']),'Праздник']=1#праздник
    return time_series

def weekseason(time_series):
    time_series['Недельная сезонность']=time_series['День недели'].map(lambda cell: week_d.loc[cell,'Недельная сезонность'])
    return time_series

time_series=setNewValues(time_series,celebrate)

#порядок дней в сезонности недельной продажи за исключением аномалий
week_d=pd.DataFrame(data=time_series[time_series['Квантили']==0].groupby('День недели')['y'].sum().sort_values())
week_d.insert(0,'Недельная сезонность',list(range(week_d.shape[0])))
for i in list(set(range(7))-set(week_d.index.values)):
    week_d.loc[i,'Недельная сезонность']=-1
    
time_series=weekseason(time_series)

mean_dict=dict(time_series.groupby(['Год','Неделя'])['y'].mean())
time_series['Среднее по неделе']=time_series.apply(lambda row: mean_dict[row['Год'],row['Неделя']] , axis=1)
mean_dict=dict(time_series.groupby(['Год'])['y'].mean())
time_series['Среднее за год']=time_series.apply(lambda row: mean_dict[row['Год']] , axis=1)
mean_dict=dict(time_series.groupby(['День года'])['y'].mean())
time_series['Среднее по дню года']=time_series.apply(lambda row: mean_dict[row['День года']] , axis=1)
time_series['Недельный тренд']=time_series['Среднее по неделе'].diff(7).fillna(0)

time_series['Среднее по неделе']=0
time_series['Среднее по дню года']=0



#Вычленяем целевую переменную
y=time_series.y
time_series.drop(['y'], axis=1, inplace=True)


#подготавливаем выборку для прогноза
time_series_forecast=weekseason(setNewValues(time_series_forecast,celebrate))
st_day=time_series_forecast.iloc[0].name
first_day_past_year=yearsago(1, st_day)
#вычисляем период которым мы должны взять из прошлого года
last_day_past_year=dt.datetime(first_day_past_year.year,12,31)
time_series[first_day_past_year:last_day_past_year]

#Сдвигаем период на год вперед
def setTimebasedValues(time_series_forecast,time_series,cols,first_day_past_year):
    #вычленяем данные с колонками
    time_series_copy=pd.DataFrame(data=time_series.loc[first_day_past_year:last_day_past_year,cols].copy())
    try:
        time_series_copy.loc[dt.datetime(first_day_past_year.year,2,28)]=time_series_copy.loc[dt.datetime(first_day_past_year.year,2,28):dt.datetime(first_day_past_year.year,2,29)].mean()
    except:
        pass
        
    time_series_forecast['Среднее за год']=time_series.tail(1)['Среднее за год'].values[0]
    
    #Если високосный год
    try:            
        time_series_forecast[cols]=pd.concat([
                time_series_copy.loc[:dt.datetime(first_day_past_year.year,2,28),[cols]].shift(366,'D'),
                time_series_copy.loc[dt.datetime(first_day_past_year.year,3,1):,[cols]].shift(365,'D')    
                ], axis=0, join='outer')
    except:
        time_series_forecast[cols]=time_series_copy[cols].shift(365,'D')
            #TODO Тут дополнительно отработать 29 февраля текущего года
    return time_series_forecast
        
time_series_forecast=setTimebasedValues(time_series_forecast,time_series,['Недельный тренд','Квантили','Среднее по неделе','Среднее по дню года'],first_day_past_year)
time_series_forecast=time_series_forecast[time_series.columns]

del week_d

## Обучение и валидация

In [6]:
next_d=dt.datetime(2017,1,1)
last_d=next_d-dt.timedelta(seconds=1)    
time_series_train=time_series.loc[:last_d]
y_train=y.loc[:last_d]

#Нормализуйте обучающую выборку с помощью класса StandardScaler
scaler = StandardScaler(with_mean=True,with_std=True)
dtrain = xgb.DMatrix(scaler.fit_transform(time_series_train), label=y_train)
#dtrain = xgb.DMatrix(time_series_train, label=y_train)

    
# задаём параметры
params = {
        'objective': 'reg:linear',
        'booster':'gblinear',
        'tree_method': 'exact',                
        'eta': 0.05,#коэффициент обучения
        'alpha': 10,
        'lambda_bias': 10,
        'eval_metric': 'rmse'
    }
trees = 1000
    
#фолды кросс-валидации
tss = TimeSeriesSplit(n_splits=10)
tss_cv=list(tss.split(time_series_train,y_train))

# прогоняем на кросс-валидации с метрикой rmse
cv = xgb.cv(params, dtrain, metrics = ('rmse'), early_stopping_rounds=True,verbose_eval=False,folds=tss_cv, show_stdv=False, num_boost_round=trees)

# обучаем xgboost с оптимальным числом деревьев, подобранным на кросс-валидации
mod_n=cv['test-rmse-mean'].argmin()
#mod_n=cv['test-mae-mean'].argmin()
bst = xgb.train(params, dtrain, num_boost_round=mod_n)
    
# запоминаем ошибку на кросс-валидации
deviation = cv.loc[mod_n]["test-rmse-mean"]
#deviation = cv.loc[mod_n]["test-mae-mean"]
prediction_test = pd.DataFrame(data=bst.predict(xgb.DMatrix(scaler.transform(time_series_forecast))),index=time_series_forecast.index)


print('Средняя ошибка на кросс-валидации: ',deviation,', номер модели: ',mod_n)
#print('Эталон: 130.7243882')

Средняя ошибка на кросс-валидации:  16.0903696 , номер модели:  155


## Валидация на кросс-обучении

In [ ]:
last=time_series.iloc[-1].name
startDt=dt.datetime(last.year,last.month,last.day)
lastDay=dt.datetime(last.year,last.month,1)-dt.timedelta(seconds=1)
startmonth=dt.datetime(lastDay.year,lastDay.month,1)

pediods=[]
for i in range(4):
    pediods.append([startmonth,startDt])    
    startDt=startmonth-dt.timedelta(seconds=1)
    startmonth=dt.datetime(startDt.year,startDt.month,1)  

pediods.append([dt.datetime(2017,5,1),dt.datetime(2017,6,30)])
    
mae=[]
mape=[]
for begin,end in reversed(pediods):
    date_div_past=begin-dt.timedelta(days=1)
        
    time_series_train=time_series.loc[:date_div_past]
    time_series_test=time_series_forecast.loc[begin:end]
    
    y_train=y.loc[:date_div_past]    
    y_test=y.loc[begin:end]    
    
    #bst = xgb.train(params, xgb.DMatrix(time_series_train, label=y_train), num_boost_round=mod_n)    
    prediction_test = pd.DataFrame(data=bst.predict(xgb.DMatrix(scaler.transform(time_series_test))),index=time_series_test.index)    
    prediction_test[0]=prediction_test[0].map(lambda val: 0 if val<0.01 else round(val,3))
    #prediction_test[0]*=1.37
        
        
    mae.append(np.mean(abs(y_test-prediction_test[0])))
    mape.append(np.mean(100-abs(100*(y_test-prediction_test[0])/y_test)))
    break    
    
print("XGBoost MAE: {} 100%-MAPE: {}%".format(round(np.mean(mae),2),round(np.mean(mape),2)))

In [ ]:
init_notebook_mode(connected = True)
trace1 = go.Scatter(
            x = prediction_test.index,
            y = prediction_test[0],
            mode = 'lines',            
            name = 'Прогноз 2017',
            line=dict(
                shape='spline',
                dash = 'dash',
                width = 4
            )
        )       

ty=(y[dt.datetime(2016,5,1):dt.datetime(2016,6,30)].shift(366,'D'))
trace0 = go.Scatter(
            x = ty.index,
            y = ty,
            mode = 'lines',            
            name = '2016 год',
            line=dict(
                shape='spline',
                dash = 'dot'
            )
        )       

ty=(y[dt.datetime(2015,5,1):dt.datetime(2015,6,30)].shift(366+365,'D'))
trace5 = go.Scatter(
            x = ty.index,
            y = ty,
            mode = 'lines',            
            name = '2015 год',
            line=dict(
                shape='spline',
                dash = 'dot'
            )
        )   


trace2 = go.Scatter(
            x = y_test.index,
            y = y_test,
            mode = 'lines',            
            name = 'Продажи 2017',
            line=dict(
                shape='spline'
            )
        ) 
 
trace3 = go.Box(
    y=prediction_test[0],
    name='Mean & SD Прогноз',    
    boxmean='sd',
    #boxpoints = 'outliers'
    boxpoints = 'all'
)

trace4 = go.Box(
    y=y_test,
    
    name='Mean & SD Данные',    
    boxmean='sd',
    #boxpoints = 'outliers',
    boxpoints = 'all'
)


fig = dict(data = [trace1,trace2,trace5,trace0])#,trace0
iplot(fig, show_link=False)

fig = dict(data = [trace4,trace3])#,trace0
iplot(fig, show_link=False)

In [ ]:
trace = go.Scatter(
    x = y_test.index,
    y = y_test-prediction_test[0]
)

data = [trace]

iplot(data)

In [3]:
df.groupby('ТоварЦеноваяГруппа')['Количество'].sum().sort_values(ascending=False)

ТоварЦеноваяГруппа
0-2100            144785
3200-5000         133622
2100-3200         115097
5000-7000         112463
7000-9000          84569
9000-12000         84432
12000-15000        65061
23000-34000        45159
15000-18000        40928
18000-23000        37653
45000-60000        12374
34000-45000        10558
<Неопределено>      8419
60000-75000         5079
90000-110000        2905
75000-90000         1928
>150000             1104
110000-150000        769
Name: Количество, dtype: int64

In [ ]:
y[dt.datetime(2016,10,1):dt.datetime(2016,10,31)].mean()/y[dt.datetime(2016,11,1):dt.datetime(2017,11,30)].mean()

In [ ]:
y[dt.datetime(2017,1,1):dt.datetime(2017,1,31)].mean()

In [ ]:
np.percentile(y, ll)

In [ ]:
dt.datetime.now()

In [ ]:
time_series['Недельный тренд']=time_series['Среднее по неделе'].diff(7).fillna(0)

In [ ]:
time_series['Недельный тренд'].plot()

In [ ]:
time_series_forecast.head(7)

In [ ]:
time_series.tail(7)

In [ ]:
prediction_test.head()

In [ ]:
y.tail()

In [ ]:
time_series_forecast['Среднее за год']=time_series.tail(1)['Среднее за год']

In [ ]:
time_series.tail(1)['Среднее за год'].values[0]